In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyhive import presto
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")
import my_func

presto_conn = presto.connect(
    host='bi-presto.serving.data.production.internal',
    port=80,
    protocol='http',
    catalog='hive',
    username='mayank.jha@rapido.bike',
    # requests_kwargs=req_kw,

)

In [4]:
start_date = '2022-01-03'
end_date = '2022-03-20'
# city = 'Kolkata'

In [75]:
# POC Segment

def get_poc(dt):
    print("fetching poc segments for : ",dt)
    orders_query = """   select captain_id, 
        city_name as city,
        segment as poc_segment,
        substr(recency_segment, 1, strpos(recency_segment, '_')-1) as recency
    from
        datasets.captain_quality_segments
        where yyyymmdd = '{sd}'      
               """.format(sd = dt.replace('-',''))
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched segment data : ", len(df_lh))

    return df_lh

# poc_segment = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) + timedelta(days=-1) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    segment = get_poc(day)
    segment['week'] = pd.to_datetime(day).strftime('%Y-%V')
    #segment['week'] = segment['week'].apply(pd.to_numeric)
    poc_segment = pd.concat([poc_segment,segment])

poc_segment.head()

fetching poc segments for :  2022-01-02
 fetched segment data :  296436
fetching poc segments for :  2022-01-09
 fetched segment data :  300116
fetching poc segments for :  2022-01-16
 fetched segment data :  295615
fetching poc segments for :  2022-01-23
 fetched segment data :  295271
fetching poc segments for :  2022-01-30
 fetched segment data :  294715
fetching poc segments for :  2022-02-06
 fetched segment data :  294632
fetching poc segments for :  2022-02-13
 fetched segment data :  295215
fetching poc segments for :  2022-02-20
 fetched segment data :  304697
fetching poc segments for :  2022-02-27
 fetched segment data :  311843
fetching poc segments for :  2022-03-06
 fetched segment data :  320984
fetching poc segments for :  2022-03-13
 fetched segment data :  0


,captain_id,city,poc_segment,recency,week
0,5ce8855625ee3218d4dddb2f,Raipur,LP_Intra_LO,active,2022-52
1,61272b0577c82b6eff385114,Raipur,LP_Inter_LO,dormant,2022-52
2,5d29f45616609737fa4b7e7b,Raipur,LP_Intra_LO,active,2022-52
3,5d5608f1bde10e0d544081bb,Raipur,ZP,churn,2022-52
4,5dd257ed010c5a46010daabc,Raipur,LP_D_MO,active,2022-52


In [73]:
poc_segment['week'].unique()

array(['2022-07', '2022-08', '2022-09', '2022-10', '2022-06', '2022-05',
       '2022-52', '2022-01', '2022-02', '2022-03', '2022-04'],
      dtype=object)

In [76]:
poc_segment.to_csv('poc_segment.csv', index=False)

In [77]:
# rf segment

def get_rf(dt):
    
    print("fetching rf segments for : ", dt)
    orders_query = """   select captainid, 
        segment as rf_segment
    from datasets.captain_link_immutable
    where day = date('{dt}')
               """.format(dt = dt)
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched segment data : ", len(df_lh))

    return df_lh

# rf_segment = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) + timedelta(days=-1) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    segment = get_rf(day)
    segment['week'] = pd.to_datetime(day).strftime('%Y-%V')
    #segment['week'] = segment['week'].apply(pd.to_numeric)
    rf_segment = pd.concat([rf_segment,segment])

rf_segment.head()

fetching rf segments for :  2022-01-02
 fetched segment data :  1290274
fetching rf segments for :  2022-01-09
 fetched segment data :  1304374
fetching rf segments for :  2022-01-16
 fetched segment data :  1316509
fetching rf segments for :  2022-01-23
 fetched segment data :  1329765
fetching rf segments for :  2022-01-30
 fetched segment data :  1344313
fetching rf segments for :  2022-02-06
 fetched segment data :  1358507
fetching rf segments for :  2022-02-13
 fetched segment data :  1370671
fetching rf segments for :  2022-02-20
 fetched segment data :  1385421
fetching rf segments for :  2022-02-27
 fetched segment data :  1401197
fetching rf segments for :  2022-03-06
 fetched segment data :  1417328
fetching rf segments for :  2022-03-13
 fetched segment data :  1435643


,captainid,rf_segment,week
0,5a5efed34d78af504953aa00,RF,2022-52
1,5c03e6dd575f2f55d2c36559,RF,2022-52
2,5c24605e4a267149c75e15f2,RF,2022-52
3,5c259bd144742d49fd441c8c,RF,2022-52
4,5c2f55ea4a267149c7658e89,RF,2022-52


In [79]:
rf_segment['week'].unique()

array(['2022-52', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05',
       '2022-06', '2022-07', '2022-08', '2022-09', '2022-10'],
      dtype=object)

In [80]:
rf_segment.to_csv('rf_segment.csv', index=False)

In [51]:
# Registration Date

def get_csv():
    
    print("fetching csv data ",)
    orders_query = """   select captainid, 
                                registrationdate, 
                                activationdate,
                                firstridedate,
                                registeredcity 
                        from datasets.captain_single_view
               """
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched csv data : ", len(df_lh))

    return df_lh


df_csv = get_csv()

df_csv.head()

fetching csv data 
 fetched csv data :  5722270


,captainid,registrationdate,activationdate,firstridedate,registeredcity
0,5bee8710fe82667f3f00295c,2018-11-16,None,None,hyderabad
1,5cb566d554bc7263ff3be96b,2020-01-23,2020-01-30,2021-09-23,chennai
2,5d943eb0a1f0a71c60538f74,2019-10-17,None,None,gorakhpur
3,5fec5d70857d1b686b21c33b,2020-12-30,2020-12-30,2020-12-31,delhi
4,60ed6f2389d307388a22f9be,2021-07-13,2021-07-14,2021-07-23,bangalore


In [52]:
df_csv['registeredcity'] = df_csv['registeredcity'].apply(lambda x : x if pd.isna(x) else x.capitalize())
df_csv['activationdate'] = df_csv['activationdate'].astype('datetime64[ns]')
df_csv['firstridedate'] = df_csv['firstridedate'].astype('datetime64[ns]')
df_csv['activation_week'] = df_csv.apply(lambda x : x.activationdate.strftime('%Y-%V') if pd.notna(x.activationdate)
                                            else x.firstridedate.strftime('%Y-%V') if pd.notna(x.firstridedate)
                                            else np.nan, axis=1)

In [53]:
df_csv.to_csv('captain_single_view.csv', index=False)

In [62]:
# Rides

def get_rides(start_date, end_date):
    
    print("fetching rides data for : ", start_date, " to : ", end_date)
    rides_query = """SELECT captain_id,
                            service_obj_city_display_name as city,
                            count(distinct order_id) as link_orders
                        from orders.order_logs_snapshot
                        where order_date >= '{sd}'
                            and order_date <= '{ed}'
                            and order_status = 'dropped'
                            and spd_fraud_flag != true
                            and service_obj_service_name = 'Link'
                            and lower(service_obj_service_name) not like '%auto%'
                            and lower(order_type) not like '%auto%'
                            
                        group by 1,2""".format(sd=start_date, ed= end_date)
    
    #print(rides_query)
    tmp = pd.read_sql(rides_query, presto_conn)
    
    print(" fetched rides for captains", len(tmp))

    return tmp

#df_rides = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]
#date_list.insert(0, (pd.to_datetime(start_date) + timedelta(days=-7)).strftime('%Y-%m-%d'))


for day in date_list : 
    rides = get_rides(day, (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'))
    rides['week'] = pd.to_datetime(day).strftime('%Y-%V')
    #rides['week'] = rides['week'].apply(pd.to_numeric)
    df_rides = pd.concat([df_rides, rides])

df_rides.head()

fetching rides data for :  2021-12-27  to :  2022-01-02
 fetched rides for captains 117193


,captain_id,city,link_orders,week
0,5fb278667e483f785bfb261e,Vishakapatnam,100,2022-04
1,5d4146d39f859318d1ef701a,Jaipur,13,2022-04
2,5d98b17061bf5a14d093b7b1,Bangalore,15,2022-04
3,5e356f003b6878aa7de915f7,Bangalore,54,2022-04
4,61a707c841b9b051c2113757,Lucknow,16,2022-04


In [78]:
df_rides['week'].unique()

array(['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-01', '2022-02', '2022-03', '2022-11', '2021-52'],
      dtype=object)

In [82]:
df_rides.to_csv('net_orders.csv', index=False)
#df_rides = pd.read_csv('net_orders.csv')

In [10]:
# Login hours

def get_login_hours(start_date, end_date):
    print("fetching Login hours for : ",start_date," to ", end_date)
    orders_query = """   SELECT 
            userid as rider, 
            CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week,
            --date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') AS logindate, 
            cast(sum(duration) as double)/cast((60*60*1000) as double) as login_hours,
            SUM(CASE WHEN quarter_hour >= '0600' and quarter_hour <= '1159' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS MORNING_DURATION,
            SUM(CASE WHEN quarter_hour >= '1200' and quarter_hour <= '1759' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS AFTERNOON_DURATION,
            SUM(CASE WHEN quarter_hour >= '1800' and quarter_hour <= '2359' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS EVENING_DURATION
        FROM hive.datasets.captain_login_hours
        WHERE yyyymmdd >= '{sd}' AND yyyymmdd <= '{ed}'
        AND status in ('2','3','6','7','8','10')
        --AND userid in (SELECT captainId from datasets.captain_single_view WHERE activationdate is not null)
        GROUP BY 1,2          
               """.format(sd = start_date.replace('-',''), ed = end_date.replace('-',''))
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched Login hours", len(df_lh))

    return df_lh

df_login_hours = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    login_hours = get_login_hours(day, (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'))
    #x = pd.to_datetime(day).strftime('%Y-%V')
    #login_hours['week'] = x[-2:]
    #login_hours['week'] = login_hours['week'].apply(pd.to_numeric)
    df_login_hours = pd.concat([df_login_hours,login_hours])

df_login_hours.head()

fetching Login hours for :  2021-10-04  to  2021-10-10


KeyboardInterrupt: 

In [6]:
df_rides = pd.read_csv('net_orders.csv')
df_csv = pd.read_csv('captain_single_view.csv')
rf_segment = pd.read_csv('rf_segment.csv')
poc_segment = pd.read_csv('poc_segment.csv')

In [7]:
poc_segment['week'] = poc_segment['week'].apply(lambda x : str(int(x[:4])+1) + '-01' if x[-2:] == '52'
                                                        else x[:5] + '0' + str(int(x[-2:])+1) if len(str(int(x[-2:])+1)) == 1 
                                                        else x[:5] + str(int(x[-2:])+1))
rf_segment['week'] = rf_segment['week'].apply(lambda x : str(int(x[:4])+1) + '-01' if x[-2:] == '52'
                                                        else x[:5] + '0' + str(int(x[-2:])+1) if len(str(int(x[-2:])+1)) == 1 
                                                        else x[:5] + str(int(x[-2:])+1))

In [8]:
poc_segment['week'] = poc_segment['week'].apply(lambda x : '2022-01' if x == '2023-01' else x)
rf_segment['week'] = rf_segment['week'].apply(lambda x : '2022-01' if x == '2023-01' else x)

In [9]:
df_final = poc_segment[(~poc_segment['captain_id'].isna()) & (poc_segment['captain_id']!='')].merge(
                                    rf_segment.rename(columns={'captainid':'captain_id'}), 
                                    how = 'left', 
                                    on = ['captain_id','week']).merge(
                                        df_rides[df_rides['week'].isin(poc_segment['week'].unique())], 
                                        how = 'outer', 
                                        on = ['captain_id','week','city']).merge(
                                            df_csv.rename(columns={'captainid':'captain_id'}),
                                            how = 'left',
                                            on = ['captain_id'])
df_final.head()

,captain_id,city,poc_segment,recency,week,rf_segment,link_orders,registrationdate,activationdate,firstridedate,registeredcity,activation_week
0,5ce8855625ee3218d4dddb2f,Raipur,LP_Intra_LO,active,2022-01,RF,13.0,2019-07-26,2019-07-26,2019-07-26,Raipur,2019-30
1,61272b0577c82b6eff385114,Raipur,LP_Inter_LO,dormant,2022-01,RF,NaN,2021-08-26,2021-08-27,2021-08-27,Raipur,2021-34
2,5d29f45616609737fa4b7e7b,Raipur,LP_Intra_LO,active,2022-01,RF,NaN,2020-12-23,2020-12-26,2021-01-11,Raipur,2020-52
3,5d5608f1bde10e0d544081bb,Raipur,ZP,churn,2022-01,CHURNED,NaN,2019-08-16,2019-10-22,2020-10-31,Raipur,2019-43
4,5dd257ed010c5a46010daabc,Raipur,LP_D_MO,active,2022-01,RF,14.0,2019-11-21,2019-11-22,2019-11-22,Raipur,2019-47


In [10]:
rides_prev = df_rides.copy()
rides_prev['week'] = rides_prev['week'].apply(lambda x : str(int(x[:4])+1) + '-01' if x[-2:] == '52'
                                                        else x[:5] + '0' + str(int(x[-2:])+1) if len(str(int(x[-2:])+1)) == 1 
                                                        else x[:5] + str(int(x[-2:])+1))
df_final = df_final.merge(rides_prev.rename(columns={"link_orders":"link_orders_prev"}), how = 'left', on = ['captain_id','week','city'])

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211006 entries, 0 to 3211005
Data columns (total 13 columns):
 #   Column            Dtype  
---  ------            -----  
 0   captain_id        object 
 1   city              object 
 2   poc_segment       object 
 3   recency           object 
 4   week              object 
 5   rf_segment        object 
 6   link_orders       float64
 7   registrationdate  object 
 8   activationdate    object 
 9   firstridedate     object 
 10  registeredcity    object 
 11  activation_week   object 
 12  link_orders_prev  float64
dtypes: float64(2), object(11)
memory usage: 343.0+ MB


In [12]:
df_final['final_segment'] = df_final.apply(lambda x : 'HH' if x.week == x.activation_week
                                                   else 'ZP' if x.poc_segment == 'ZP'
                                                   else 'Resurrected' if pd.notna(x.link_orders) and pd.isna(x.link_orders_prev)
                                                   else 'No_Segment' if pd.isna(x.poc_segment)
                                                   else 'Inter' if x.poc_segment.find('Inter')!=-1 
                                                   else 'HP_UHP_D' if x.poc_segment[:x.poc_segment.find('_')] in ('HP','UHP') and x.poc_segment.find('_D') != -1
                                                   else 'MP_LP_D' if x.poc_segment[:x.poc_segment.find('_')] in ('MP','LP') and x.poc_segment.find('_D') != -1
                                                   else 'HP_UHP_Intra' if x.poc_segment[:x.poc_segment.find('_')] in ('HP','UHP') and x.poc_segment.find('_Intra') != -1
                                                   else 'MP_LP_Intra' if x.poc_segment[:x.poc_segment.find('_')] in ('MP','LP') and x.poc_segment.find('_Intra') != -1
                                                   else 'No_Segment',
                                                   axis = 1)

In [13]:
df_final[(df_final['final_segment']=='Resurrected') & (df_final['week']=='2022-01') & (df_final['poc_segment'].isna())]

,captain_id,city,poc_segment,recency,week,rf_segment,link_orders,registrationdate,activationdate,firstridedate,registeredcity,activation_week,link_orders_prev,final_segment
3158877,6103b3026c5e5f74cac02098,Delhi,NaN,NaN,2022-01,NaN,2.0,2021-08-30,2021-08-31,2021-08-31,Delhi,2021-35,NaN,Resurrected
3158878,5d03a6e5abbf48476381c08f,Hyderabad,NaN,NaN,2022-01,NaN,16.0,2019-08-10,2019-08-13,2019-08-13,Hyderabad,2019-33,NaN,Resurrected
3158879,5d387f81c6977b6562c664da,Chennai,NaN,NaN,2022-01,NaN,30.0,2019-07-24,2020-10-15,2020-10-15,Chennai,2020-42,NaN,Resurrected
3158883,61cdd5c1b387073c44d8a776,Bangalore,NaN,NaN,2022-01,NaN,18.0,2021-12-30,2022-01-02,2022-01-04,Bangalore,2022-52,NaN,Resurrected
3158886,61290d8634b93d0c7433401e,Vishakapatnam,NaN,NaN,2022-01,NaN,22.0,2021-08-31,2021-08-31,2021-09-07,Vishakapatnam,2021-35,NaN,Resurrected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177750,5d29e62a16609737fa4b61be,Bangalore,NaN,NaN,2022-01,NaN,1.0,2019-07-18,2019-07-31,2019-07-31,Bangalore,2019-31,NaN,Resurrected
3177751,5bb078de57433b551d16d98e,Delhi,NaN,NaN,2022-01,NaN,1.0,2020-01-13,2020-01-14,2021-07-15,Delhi,2020-03,NaN,Resurrected
3177752,5f93d36d3fbcbe3c5849c7e9,Hyderabad,NaN,NaN,2022-01,NaN,1.0,2021-04-18,2021-07-06,2021-07-06,Hyderabad,2021-27,NaN,Resurrected
3177753,61aa482bf3de6f0738419a50,Amritsar,NaN,NaN,2022-01,NaN,2.0,2021-12-03,2021-12-04,2022-01-03,Amritsar,2021-48,NaN,Resurrected


In [17]:
#df_final['final_segment'] = df_final['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])

In [14]:
# Gross captains

df_final.pivot_table(index='week', columns = 'final_segment', values = 'captain_id', aggfunc = 'nunique')

final_segment,HH,HP_UHP_D,Inter,MP_LP_D,MP_LP_Intra,No_Segment,Resurrected,ZP
week,,,,,,,,
2022-01,8336.0,21411.0,27235.0,123051.0,27142.0,NaN,24590.0,83255.0
2022-02,6778.0,20304.0,27870.0,125571.0,28369.0,NaN,21316.0,85126.0
2022-03,7682.0,18213.0,27221.0,120350.0,28219.0,NaN,25944.0,85469.0
2022-04,8447.0,16996.0,26984.0,119429.0,27287.0,NaN,26987.0,88062.0
2022-05,8201.0,15543.0,27310.0,120046.0,27841.0,NaN,25564.0,89471.0
2022-06,7794.0,15645.0,26746.0,120650.0,27432.0,NaN,25237.0,90233.0
2022-07,8242.0,15989.0,25915.0,120800.0,27154.0,1117.0,27315.0,90391.0
2022-08,9660.0,18548.0,25822.0,124203.0,27302.0,NaN,28049.0,93151.0
2022-09,10244.0,21367.0,25924.0,126681.0,27316.0,NaN,28796.0,94936.0


In [89]:
# Net captains

df_final[df_final['link_orders']>0].pivot_table(index='week', columns = 'final_segment', values = 'captain_id', aggfunc = 'nunique')

final_segment,HH,HP_UHP_D,Inter,MP_LP_D,MP_LP_Intra,No_Segment,Resurrected,ZP
week,,,,,,,,
2022-01,8331.0,12573.0,2436.0,51346.0,9006.0,NaN,24590.0,5703.0
2022-02,6771.0,11226.0,2409.0,47936.0,8614.0,NaN,21316.0,5191.0
2022-03,7675.0,9727.0,2409.0,45185.0,8607.0,NaN,25944.0,5742.0
2022-04,8438.0,9004.0,2710.0,48406.0,9106.0,NaN,26987.0,6174.0
2022-05,8198.0,8084.0,3127.0,51478.0,10269.0,NaN,25564.0,6344.0
2022-06,7792.0,8542.0,2848.0,52528.0,10360.0,NaN,25237.0,6413.0
2022-07,8242.0,9509.0,2968.0,53291.0,10392.0,1117.0,27315.0,6585.0
2022-08,9660.0,12280.0,2860.0,58000.0,10709.0,NaN,28049.0,6731.0
2022-09,10243.0,14149.0,3234.0,60305.0,11337.0,NaN,28796.0,7232.0


In [103]:
# Captains doing 0 rides

df_final[(df_final['link_orders']==0) | (df_final['link_orders'].isna())].pivot_table(index='week', columns = 'final_segment', values = 'captain_id', aggfunc = 'nunique')

final_segment,HH,HP_UHP_D,Inter,MP_LP_D,MP_LP_Intra,ZP
week,,,,,,
2022-01,5.0,8844.0,24799.0,71733.0,18136.0,77555.0
2022-02,7.0,9084.0,25461.0,77663.0,19756.0,79936.0
2022-03,7.0,8489.0,24812.0,75182.0,19612.0,79731.0
2022-04,9.0,7992.0,24274.0,71046.0,18181.0,81891.0
2022-05,3.0,7460.0,24184.0,68595.0,17572.0,83128.0
2022-06,2.0,7110.0,23898.0,68161.0,17072.0,83821.0
2022-07,NaN,6485.0,22948.0,67548.0,16763.0,83808.0
2022-08,NaN,6274.0,22962.0,66232.0,16593.0,86422.0
2022-09,1.0,7223.0,22690.0,66407.0,15981.0,87707.0


In [90]:
# RPR

df_final.pivot_table(index='week', columns = 'final_segment', values = 'link_orders', aggfunc = 'mean')

final_segment,HH,HP_UHP_D,Inter,MP_LP_D,MP_LP_Intra,No_Segment,Resurrected,ZP
week,,,,,,,,
2022-01,8.926850,37.819057,5.950328,13.636369,5.020986,NaN,7.741705,6.623707
2022-02,8.851431,37.889364,5.375675,13.510159,5.033202,NaN,6.564901,5.736467
2022-03,8.950826,35.539529,5.156912,13.808658,5.404554,NaN,7.445023,6.091606
2022-04,8.883919,33.469014,5.352030,13.323348,5.361300,NaN,6.942312,5.823615
2022-05,10.060894,39.514349,5.519028,14.788842,5.791411,NaN,7.839532,6.265605
2022-06,10.417907,44.287169,5.893258,16.338201,5.808108,NaN,8.246147,6.933416
2022-07,11.152746,46.550836,6.494946,17.189638,6.148287,16.47359,8.629255,7.165983
2022-08,10.461355,46.319925,6.053846,16.460509,5.560183,NaN,8.563703,7.099391
2022-09,11.305387,45.910453,6.971861,16.875738,5.960660,NaN,8.958199,7.712666


In [93]:
# Weekly Net Orders

df_final.pivot_table(index='week', columns = 'final_segment', values = 'link_orders', aggfunc = 'sum')

final_segment,HH,HP_UHP_D,Inter,MP_LP_D,MP_LP_Intra,No_Segment,Resurrected,ZP
week,,,,,,,,
2022-01,74441.0,475499.0,14495.0,700173.0,45219.0,NaN,190384.0,37775.0
2022-02,59995.0,425346.0,12950.0,647623.0,43356.0,NaN,139944.0,29778.0
2022-03,68805.0,345693.0,12423.0,623958.0,46517.0,NaN,193176.0,34978.0
2022-04,75078.0,301355.0,14504.0,644930.0,48820.0,NaN,187373.0,35955.0
2022-05,82610.0,319434.0,17258.0,761300.0,59472.0,NaN,200449.0,39749.0
2022-06,81218.0,378301.0,16784.0,858213.0,60172.0,NaN,208108.0,44464.0
2022-07,91999.0,442745.0,19277.0,916053.0,63893.0,18401.0,235734.0,47188.0
2022-08,101109.0,568855.0,17314.0,954726.0,59544.0,NaN,240229.0,47786.0
2022-09,115835.0,649587.0,22547.0,1017742.0,67576.0,NaN,258023.0,55778.0


In [104]:
# Segment Retention

df_ret = pd.DataFrame()

for s in df_final['final_segment'].unique():
    for w in df_final['week'].unique():
        if w == 52:
            wx = str(int(w[:4])+1) + '-01'
            base = df_final[(df_final['week']==w) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            forward = df_final[(df_final['week']==wx) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
        else :
            wx = w[:5] + '0' + str(int(w[-2:])+1) if len(str(int(w[-2:])+1)) == 1 else w[:5] + str(int(w[-2:])+1)
            base = df_final[(df_final['week']==w) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            forward = df_final[(df_final['week']==wx) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
df_ret.to_csv('x.csv')

In [105]:
# Segment Movement

df_movement = pd.DataFrame()

for s in df_final['final_segment'].unique():
    for w in df_final['week'].unique():
        if w[-2:] == '52':
            wx = str(int(w[:4])+1) + '-01'
            base = df_final[(df_final['week']==w) & (df_final['final_segment']!=s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            forward = df_final[(df_final['week']==wx) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_movement = pd.concat([df_movement, x])
        else:
            wx = w[:5] + '0' + str(int(w[-2:])+1) if len(str(int(w[-2:])+1)) == 1 else w[:5] + str(int(w[-2:])+1)
            base = df_final[(df_final['week']==w) & (df_final['final_segment']!=s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            forward = df_final[(df_final['week']==wx) & (df_final['final_segment']==s) & (df_final['link_orders']>0) & (~df_final['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_movement = pd.concat([df_movement, x])
df_movement.to_csv('x.csv')

In [14]:
tmp = df_final[(df_final['city'].isin(['Hyderabad','Bangalore','Chennai','Jaipur','Kolkata','Delhi'])) & (df_final['link_orders']>0)]

#tmp.pivot_table(index='week', columns = 'final_segment', values = 'link_orders', aggfunc = 'sum')

In [16]:
df_ret = pd.DataFrame()

for s in tmp['final_segment'].unique():
    for w in tmp['week'].unique():
        if w == 52:
            wx = str(int(w[:4])+1) + '-01'
            base = tmp[(tmp['week']==w)  & (tmp['final_segment']==s) & (tmp['link_orders']>0) & (~tmp['link_orders'].isna())]
            forward = tmp[(tmp['week']==wx) & (tmp['link_orders']>0) & (~tmp['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
        else :
            wx = w[:5] + '0' + str(int(w[-2:])+1) if len(str(int(w[-2:])+1)) == 1 else w[:5] + str(int(w[-2:])+1)
            base = tmp[(tmp['week']==w) & (tmp['final_segment']==s) & (tmp['link_orders']>0) & (~tmp['link_orders'].isna())]
            forward = tmp[(tmp['week']==wx) & (tmp['link_orders']>0) & (~tmp['link_orders'].isna())]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
            x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
df_ret#.to_csv('x.csv')

,segment,week,retention
0,MP_LP_D,2022-02,68.679762
0,MP_LP_D,2022-03,71.235035
0,MP_LP_D,2022-04,74.983547
0,MP_LP_D,2022-05,74.427147
0,MP_LP_D,2022-06,73.637199
...,...,...,...
0,No_Segment,2022-07,0.000000
0,No_Segment,2022-08,56.451613
0,No_Segment,2022-09,0.000000
0,No_Segment,2022-10,0.000000


In [20]:
# Rides

def get_rides(start_date, end_date):
    
    print("fetching rides data for : ", start_date, " to : ", end_date)
    rides_query = """SELECT captain_id,
                            service_obj_city_display_name as city,
                            sum(case when service_obj_service_name = 'Link' then 1 else 0 end) as link_orders,
                            sum(case when service_obj_service_name = 'Delivery' then 1 else 0 end) as delivery_orders

                        from orders.order_logs_snapshot
                        where order_date >= '{sd}'
                            and order_date <= '{ed}'
                            and order_status = 'dropped'
                            and spd_fraud_flag != true
                            --and service_obj_service_name = 'Link'
                            and lower(service_obj_service_name) not like '%auto%'
                            and lower(order_type) not like '%auto%'
                            
                        group by 1,2""".format(sd=start_date, ed= end_date)
    
    #print(rides_query)
    tmp = pd.read_sql(rides_query, presto_conn)
    
    print(" fetched rides for captains", len(tmp))

    return tmp

df_orders = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]
#date_list.insert(0, (pd.to_datetime(start_date) + timedelta(days=-7)).strftime('%Y-%m-%d'))


for day in date_list : 
    rides = get_rides(day, (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'))
    rides['week'] = pd.to_datetime(day).strftime('%Y-%V')
    #rides['week'] = rides['week'].apply(pd.to_numeric)
    df_orders = pd.concat([df_orders, rides])

df_orders.head()

fetching rides data for :  2022-01-03  to :  2022-01-09
 fetched rides for captains 130979
fetching rides data for :  2022-01-10  to :  2022-01-16
 fetched rides for captains 121705
fetching rides data for :  2022-01-17  to :  2022-01-23
 fetched rides for captains 125660
fetching rides data for :  2022-01-24  to :  2022-01-30
 fetched rides for captains 131482
fetching rides data for :  2022-01-31  to :  2022-02-06
 fetched rides for captains 130913
fetching rides data for :  2022-02-07  to :  2022-02-13
 fetched rides for captains 130697
fetching rides data for :  2022-02-14  to :  2022-02-20
 fetched rides for captains 136922
fetching rides data for :  2022-02-21  to :  2022-02-27
 fetched rides for captains 145267
fetching rides data for :  2022-02-28  to :  2022-03-06
 fetched rides for captains 152357
fetching rides data for :  2022-03-07  to :  2022-03-13
 fetched rides for captains 161333
fetching rides data for :  2022-03-14  to :  2022-03-20
 fetched rides for captains 157013

,captain_id,city,link_orders,delivery_orders,week
0,5dea58833858de338017043e,Bangalore,95,8,2022-01
1,61141227f3519d4ef04079c1,Chennai,40,0,2022-01
2,618caba5faa41b0ffacbbc67,Prayagraj,18,0,2022-01
3,61587ebc77c7e6d35a8f88ff,Mumbai,13,0,2022-01
4,5f6d9e2b4df9b48b12418caf,Trichy,15,0,2022-01


In [22]:
df_final = poc_segment[(~poc_segment['captain_id'].isna()) & (poc_segment['captain_id']!='')].merge(
                                    rf_segment.rename(columns={'captainid':'captain_id'}), 
                                    how = 'left', 
                                    on = ['captain_id','week']).merge(
                                        df_orders[df_orders['week'].isin(poc_segment['week'].unique())], 
                                        how = 'outer', 
                                        on = ['captain_id','week','city']).merge(
                                            df_csv.rename(columns={'captainid':'captain_id'}),
                                            how = 'left',
                                            on = ['captain_id'])


# rides_prev = df_orders.copy()
# rides_prev['week'] = rides_prev['week'].apply(lambda x : str(int(x[:4])+1) + '-01' if x[-2:] == '52'
#                                                         else x[:5] + '0' + str(int(x[-2:])+1) if len(str(int(x[-2:])+1)) == 1 
#                                                         else x[:5] + str(int(x[-2:])+1))
# df_final = df_final.merge(rides_prev.rename(columns={"link_orders":"link_orders_prev"}), how = 'left', on = ['captain_id','week','city'])


# df_final['final_segment'] = df_final.apply(lambda x : 'HH' if x.week == x.activation_week
#                                                    else 'ZP' if x.poc_segment == 'ZP'
#                                                    else 'Resurrected' if pd.notna(x.link_orders) and pd.isna(x.link_orders_prev)
#                                                    else 'No_Segment' if pd.isna(x.poc_segment)
#                                                    else 'Inter' if x.poc_segment.find('Inter')!=-1 
#                                                    else 'HP_UHP_D' if x.poc_segment[:x.poc_segment.find('_')] in ('HP','UHP') and x.poc_segment.find('_D') != -1
#                                                    else 'MP_LP_D' if x.poc_segment[:x.poc_segment.find('_')] in ('MP','LP') and x.poc_segment.find('_D') != -1
#                                                    else 'HP_UHP_Intra' if x.poc_segment[:x.poc_segment.find('_')] in ('HP','UHP') and x.poc_segment.find('_Intra') != -1
#                                                    else 'MP_LP_Intra' if x.poc_segment[:x.poc_segment.find('_')] in ('MP','LP') and x.poc_segment.find('_Intra') != -1
#                                                    else 'No_Segment',
#                                                    axis = 1)

In [70]:
tmp = df_final[(df_final['city'].isin(['Chennai']))]

#tmp.pivot_table(index='week', columns = 'final_segment', values = 'link_orders', aggfunc = 'sum')

In [71]:
df_ret = pd.DataFrame()

for w in tmp['week'].unique():
    if w == 52:
        wx = str(int(w[:4])+1) + '-01'
        base = tmp[(tmp['week']==w) & (tmp['link_orders']>0) & (tmp['delivery_orders']>0)]
        forward = tmp[(tmp['week']==wx) & ((tmp['link_orders']>0) | (tmp['delivery_orders']>0))]
        ret = base.merge(forward, how = 'inner', on = 'captain_id')
        retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
        x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention], "base_captain" : [base['captain_id'].nunique()],"retained_captain" : [ret['captain_id'].nunique()]})
        df_ret = pd.concat([df_ret, x])
    else :
        wx = w[:5] + '0' + str(int(w[-2:])+1) if len(str(int(w[-2:])+1)) == 1 else w[:5] + str(int(w[-2:])+1)
        base = tmp[(tmp['week']==w) & (tmp['link_orders']>0) & (tmp['delivery_orders']>0)]
        forward = tmp[(tmp['week']==wx) & ((tmp['link_orders']>0) | (tmp['delivery_orders']>0))]
        ret = base.merge(forward, how = 'inner', on = 'captain_id')
        retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0 if base['captain_id'].nunique() !=0 else 0
        x = pd.DataFrame({"segment" : [s], "week" : [wx], "retention" : [retention], "base_captain" : [base['captain_id'].nunique()],"retained_captain" : [ret['captain_id'].nunique()]})
        df_ret = pd.concat([df_ret, x])
df_ret#.to_csv('x.csv')

,segment,week,retention,base_captain,retained_captain
0,No_Segment,2022-02,70.090347,2103,1474
0,No_Segment,2022-03,76.643074,1978,1516
0,No_Segment,2022-04,78.749389,2047,1612
0,No_Segment,2022-05,76.733977,2278,1748
0,No_Segment,2022-06,77.975633,2134,1664
0,No_Segment,2022-07,77.597672,2406,1867
0,No_Segment,2022-08,79.154458,2389,1891
0,No_Segment,2022-09,76.850181,2216,1703
0,No_Segment,2022-10,75.405405,2220,1674
0,No_Segment,2022-11,0.000000,1940,0


In [74]:
df_final[(df_final['week']=='2022-02') & (df_final['link_orders']>0) & (df_final['city'].isin(['Hyderabad','Bangalore','Chennai','Jaipur','Kolkata','Delhi']))]['captain_id'].nunique()
                                                                        

63214

In [123]:
df_movement.to_csv('x.csv')

In [126]:
poc_segment[poc_segment['week']==48].to_csv('x.csv', index=False)

In [129]:
df_final[df_final['week']==48].to_csv('x.csv', index=False)

In [132]:
df_rides[(df_rides['week']==1) & (df_rides['link_orders']>0)]['captain_id'].nunique()

113980

In [136]:
poc_segment[poc_segment['week']==1].to_csv('x.csv')

In [137]:
df_rides[df_rides['week']==1].to_csv('x.csv')

In [43]:
# Pan India LH by segment

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_login_hours.rename(columns={'rider':'captain_id'}), how = 'outer', on = ['captain_id','week'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='final_segment', columns = 'week', values = 'login_hours', aggfunc = 'mean')

week,1,2,3,40,41,42,43,44,45,46,47,48,49,50,51,52
final_segment,,,,,,,,,,,,,,,,
HH,16.533023,17.510240,19.734976,12.735990,10.958780,11.322616,11.905275,10.758116,11.315912,10.399669,10.696207,11.534917,11.307547,12.008399,13.382549,14.338876
HP,22.604303,22.318412,21.511753,24.262604,21.700752,20.152932,24.521624,22.874514,28.012807,20.741760,23.224352,22.703558,22.379755,23.343796,23.734191,21.769160
Inter_Intra,7.621627,7.762175,8.113884,6.439241,5.749454,6.563487,7.965378,6.217386,6.736846,6.872951,7.232155,6.756270,7.118063,7.394573,7.834614,7.924639
LP,10.308043,10.370338,10.731783,11.158404,9.982352,8.932384,11.102304,8.806383,9.777703,9.375329,9.930661,9.404818,9.880309,10.318763,10.662920,10.486760
MP,14.814402,14.990934,15.470176,17.166937,15.006409,13.222823,15.922759,13.693280,16.866011,14.337317,15.582030,15.028878,15.061990,15.380288,15.770147,15.075546
UHP,31.706611,32.297664,27.098075,33.080446,29.132266,28.198715,35.600938,31.887833,37.620778,26.304509,30.386086,31.036547,30.428520,34.633565,35.468633,28.927065
ZP,4.820721,4.784514,4.825998,3.931621,3.519194,4.126914,4.881337,4.122140,3.789916,3.929164,4.097265,3.893283,4.192823,4.706497,5.093554,4.933285


In [44]:
# Pan India Average LH by segment

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_login_hours.rename(columns={'rider':'captain_id'}), how = 'outer', on = ['captain_id','week'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='week', values = 'login_hours', aggfunc = 'mean')

,login_hours
week,
1,13.973225
2,14.502331
3,15.841945
40,11.546130
41,10.079553
42,10.920382
43,12.199544
44,10.767450
45,11.139910


In [45]:
# Pan India RPR by segment

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'outer', on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='final_segment', columns = 'week', values = 'link_orders', aggfunc = 'mean')

week,1,2,3,40,41,42,43,44,45,46,47,48,49,50,51,52
final_segment,,,,,,,,,,,,,,,,
HH,8.545609,8.006128,8.311905,9.756596,9.096204,8.695290,8.829220,7.077810,8.299153,9.361421,9.462068,9.317057,10.136226,9.240827,8.732824,8.577977
HP,31.065540,31.086413,30.229939,35.500089,31.072632,30.041210,33.753580,34.383946,46.049936,33.225946,35.105885,35.028780,34.528688,33.948160,32.160655,30.464221
Inter_Intra,4.918567,4.663561,4.956839,8.634405,8.021686,4.764733,5.880201,4.204862,4.907932,5.464010,5.413856,5.155155,5.592115,5.050310,4.757885,5.298910
LP,9.524800,9.323972,9.367913,9.196035,8.371242,8.833362,10.761608,8.977755,11.072852,10.480990,10.845545,10.241031,10.707598,10.464675,9.385173,9.603405
MP,16.930482,16.812069,17.433574,20.193284,17.634240,16.772291,18.612453,17.439390,23.544335,19.376648,20.034725,19.434905,19.408106,18.642532,17.321313,17.257136
UHP,51.886107,54.106490,45.172849,55.982609,47.701843,48.368826,55.421938,54.154167,65.724944,47.735701,52.037660,54.172835,53.795760,57.527488,56.609886,47.779273
ZP,6.621977,5.736133,6.091606,6.861421,6.677338,5.712403,7.296311,4.963842,6.021295,7.150788,7.482993,7.256424,8.334687,7.922503,6.827781,6.723256


In [47]:
# Pan India average RPR by segment

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'outer', on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='week', values = 'link_orders', aggfunc = 'mean')

,link_orders
week,
1,13.491741
2,13.134067
3,12.587940
40,13.195630
41,12.249512
42,12.734269
43,14.350129
44,13.854149
45,15.964336


In [27]:
# Pan India Net Orders by segment

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'outer', on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='final_segment', columns = 'week', values = 'link_orders', aggfunc = 'sum')

week,1,2,3,40,41,42,43,44,45,46,47,48,49,50,51,52
final_segment,,,,,,,,,,,,,,,,
HH,161324.0,124111.0,147523.0,139773.0,104479.0,141055.0,161716.0,82230.0,132247.0,165201.0,179978.0,181459.0,220767.0,213306.0,191048.0,156814.0
HP,327058.0,288513.0,252026.0,200185.0,178823.0,230356.0,280526.0,276722.0,324606.0,245872.0,303034.0,331057.0,350846.0,379167.0,380750.0,337178.0
Inter_Intra,76346.0,70584.0,78898.0,204031.0,170525.0,61527.0,84375.0,50164.0,65089.0,82665.0,85566.0,76054.0,84961.0,77497.0,73162.0,89435.0
LP,323767.0,297817.0,303862.0,212925.0,174013.0,205994.0,271171.0,194530.0,276622.0,276216.0,312612.0,293170.0,322984.0,331343.0,309401.0,331692.0
MP,441784.0,397286.0,393804.0,313319.0,255961.0,359665.0,430748.0,356932.0,424575.0,361452.0,410211.0,422165.0,448696.0,467070.0,458703.0,457038.0
UHP,169928.0,150903.0,114466.0,57942.0,56956.0,75407.0,95492.0,103976.0,118042.0,103491.0,129886.0,150763.0,159881.0,198815.0,213023.0,166893.0
ZP,37785.0,29782.0,34978.0,31391.0,26489.0,12573.0,18197.0,9198.0,28559.0,39930.0,44000.0,41514.0,51300.0,51013.0,43452.0,43365.0


In [48]:
# Segment Retention

df_ret = pd.DataFrame()

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'inner', on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])

for s in df_tmp['final_segment'].unique():
    for w in df_tmp['week'].unique():
        if w == 52:
            base = df_tmp[(df_tmp['week']==w) & (df_tmp['final_segment']==s)]
            forward = df_tmp[(df_tmp['week']==1)]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
            x = pd.DataFrame({"segment" : [s], "week" : [w], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
        else :
            base = df_tmp[(df_tmp['week']==w) & (df_tmp['final_segment']==s)]
            forward = df_tmp[(df_tmp['week']==w+1)]
            ret = base.merge(forward, how = 'inner', on = 'captain_id')
            retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
            x = pd.DataFrame({"segment" : [s], "week" : [w], "retention" : [retention]})
            df_ret = pd.concat([df_ret, x])
df_ret.to_csv('x.csv', index=False)

In [50]:
# Overall Retention

df_ret = pd.DataFrame()

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'inner', on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])

for w in df_tmp['week'].unique():
    if w == 52:
        base = df_tmp[(df_tmp['week']==w)]
        forward = df_tmp[(df_tmp['week']==1)]
        ret = base.merge(forward, how = 'inner', on = 'captain_id')
        retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
        x = pd.DataFrame({"week" : [w], "retention" : [retention]})
        df_ret = pd.concat([df_ret, x])
    else :
        base = df_tmp[(df_tmp['week']==w)]
        forward = df_tmp[(df_tmp['week']==w+1)]
        ret = base.merge(forward, how = 'inner', on = 'captain_id')
        retention = ret['captain_id'].nunique()/base['captain_id'].nunique()*100.0
        x = pd.DataFrame({"week" : [w], "retention" : [retention]})
        df_ret = pd.concat([df_ret, x])
df_ret.to_csv('x.csv', index=False)

In [40]:
# Pan India Net captains by segment

# poc_segment_copy = poc_segment.copy()
# poc_segment_copy['week'] = poc_segment_copy['week'].apply(lambda x : 52 if x == 1 else x-1)

df_tmp = poc_segment[~poc_segment['poc_segment'].isna()].merge(df_rides[df_rides['link_orders']>0], how = 'outer', left_on = ['captain_id','week','city'], right_on = ['captain_id','week','city'])
df_tmp['poc_segment'] = df_tmp['poc_segment'].apply(lambda x : 'HH' if pd.isna(x) else x)
df_tmp['final_segment'] = df_tmp['poc_segment'].apply(lambda x : x if x.find('_')==-1 else 'Inter_Intra' if x.find('Int')!=-1 else x[:x.find('_')])
df_tmp.pivot_table(index='final_segment', columns = 'week', values = 'captain_id', aggfunc = 'nunique')

week,1,2,3,40,41,42,43,44,45,46,47,48,49,50,51,52
final_segment,,,,,,,,,,,,,,,,
HH,18868,15494,17736,14325,11484,16218,18308,11612,15920,17621,18995,19448,21737,23041,21833,18278
HP,16758,15956,14655,8009,8511,12806,13230,13663,11270,11858,13041,14211,15445,16504,17550,17257
Inter_Intra,58410,60317,60322,78898,80208,43593,43611,44957,45058,47868,48819,48248,49934,51403,54216,56981
LP,83232,85273,84573,45465,45913,56206,55744,56075,60816,61768,63854,64769,68525,72709,76463,80120
MP,48596,47972,45648,23827,24083,42081,42431,42208,34297,34475,35867,37422,40409,43906,46893,47955
UHP,5889,5196,4704,1506,1703,2573,2748,3050,2678,3415,3982,4482,5158,5725,6366,6300
ZP,83259,85128,85471,61882,62446,23658,23952,24158,62157,64509,67418,70656,75090,79016,80329,82187


In [36]:
df_rides.pivot_table(index='week', values = 'link_orders', aggfunc = 'sum')

,link_orders
week,
1,1537991
2,1358995
3,1325573
40,1159566
41,967246
42,1086577
43,1342225
44,1073752
45,1369740


In [38]:
df_rides[df_rides['week']==52].to_csv('x.csv')

In [54]:
rides = df_rides[(df_rides['week'].isin(1,2,3)) & (df_rides['link_orders']>0) & (df_rides['city'].isin('Hyderabad','Jaipur'))]

,captain_id,week,city,link_orders
0,5c2c967644742d49fd467303,40,Delhi,14
1,5e68606159c6ddc839571796,40,Chandigarh,79
2,5dddf3995ede7607ced5ba8f,40,Delhi,8
3,5c1a00ceb88aae47f8378a21,40,Vijayawada,30
4,5d29431706ee2c37f1f50d53,40,Ranchi,7
...,...,...,...,...
215367,5aeea043a28f646244fbb587,3,Hyderabad,2
215368,61122f528dddcbef0c6b6a94,3,Bhopal,1
215369,612f11f99db16b1aeeee1a4a,3,Pune,0
215370,602664f319e49870e11d159f,3,Hyderabad,0
